In [9]:
# -*- coding：utf-8 -*-

import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import random
import numpy as np
import time
import torch
import torch.nn
import generate_networks
import argparse
from sklearn.preprocessing import normalize
import openpyxl
import send_mail as sm
from sklearn.model_selection import KFold
import torch.backends.cudnn as cudnn
from time import strftime
from time import gmtime
from time import sleep
from tqdm import tqdm
import torch.nn.functional as F

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer

# 数据集路径
path = "D:\\Pycharm\\Project\\ML\\Dataset\\DEAP\\data_preprocessed_python\\DEAP1s\\"

def readData(way=''): # 读取数据
    with open(path + 'data.npy', 'rb') as filedata:
        data = np.load(filedata)
    with open(path + 'label.npy', 'rb') as filelabel:
        label = np.load(filelabel)

    end = 30 * 40 * 60
    data = data[:end, :]
    label = label[:end, [0]]

    if way != '':
        if way == 'StandardScaler':
            scaler = StandardScaler()
        elif way == 'Normalizer':
            scaler = Normalizer()
        elif way == 'MinMaxScaler':
            scaler = MinMaxScaler()
        data = scaler.fit_transform(data)

    for i in range(len(label)):
        if label[i] < 5:
            label[i] = 0
        else:
            label[i] = 1

    label = torch.tensor(label, dtype=torch.double).cuda()
    data = torch.tensor(data, dtype=torch.double).cuda()
    print("data_shape:{0}, label_shape:{1}".format(data.shape, label.shape))

    return data, label


In [11]:
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix


def Res(prediction, test_y):
    r = []
    t = confusion_matrix(test_y, prediction)
    r.append(t[0][0])
    r.append(t[0][1])
    r.append(t[1][0])
    r.append(t[1][1])
    r.append(metrics.accuracy_score(test_y, prediction))
    r.append(metrics.f1_score(test_y, prediction))
    r.append(metrics.precision_score(test_y, prediction))
    r.append(metrics.recall_score(test_y, prediction))
    r.append(metrics.roc_auc_score(test_y, prediction))

#     print(confusion_matrix(test_y, prediction))
    return r


def res1(prediction, label_test):
    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(len_test):
        if (prediction[i] > 0.5 and label_test[i] == 1):
            tp += 1
        elif (prediction[i] <= 0.5 and label_test[i] == 0):
            tn += 1
        elif (prediction[i] > 0.5 and label_test[i] == 0):
            fp += 1
        else:
            fn += 1

    result = [tp, tn, fp, fn]
    result.append(round(result[0] / (result[0] + result[2]), 6))  # precision
    result.append(round(result[0] / (result[0] + result[3]), 6))  # recall
    result.append(round(result[2] / (result[2] + result[3]), 6))  # specificity
    result.append(round((result[0] + result[1]) / (sum(result[0:4])), 6))  # accuracy
    result.append(round(2 * result[5] * result[4] / (result[5] + result[4]), 6))  # f1-score
    print("My result:",result)


In [12]:
def sigmoid_(x, a, k):  # sigmoid = tanh(ax) * k
    t = torch.nn.Tanh()
    ans = t(a*x)
    ans *= k
    return ans


def correct_dimensions(s, targetlength):
    if s is not None:
        s = np.array(s)
        if s.ndim == 0:
            s = np.array([s] * targetlength)
        elif s.ndim == 1:
            if not len(s) == targetlength:
                raise ValueError("arg must have length " + str(targetlength))
        else:
            raise ValueError("Invalid argument")
    return s


def identity(x):
    # f = F.softmax(x, dim=0)
    x = x.cpu().numpy()
    arr = np.full(x.shape, True, dtype=bool)
    np.greater(x, 0.5, out=arr)

    x[:] = 0
    np.putmask(x, arr, 1)

    return x


def step_function(x):
    if x > 0.5:
        return 1
    else:
        return 0


def backward(x, a):
    '''
    # 求不同系数a下，tanh导数
    '''
    up = torch.exp(-2 * a * x) * (a - 1) + 3 * a + 1
    down = ( torch.exp(a * x) + torch.exp(-a * x)) ** 2
    return up / down

def backward1(x, a):
    ans = x.mul(a)
    return ans


def sigmoid(x):
    # return 1 / (1 + torch.exp(-1 * x))
    # return 1 / (1 + torch.exp(-10 * (x - 1))) # 模块化论文中使用的
    # lr = torch.nn.Sigmoid()
    lr = torch.nn.Tanh()
    # lr = torch.nn.LeakyReLU(0.1)
    # lr = torch.nn.ReLU()
    # lr = torch.nn.RReLU(0.1, 0.3)
    # x = torch.where(x > 0.5, 1, x)
    # x = torch.where(x <= 0.5, 0, x)
    # i = 0
    # while i != len(x):
    #     a = float(np.random.lognormal(0, 1, 1))
    #     Derivative = backward(x[i], a)
    #     if Derivative < 0:
    #         print(i, ": ", Derivative)
    #         continue
    #     else:
    #         x[i] = x[i] * a
    #         i += 1

    # print(x)

    # log_normal = torch.from_numpy(log_normal)
    # log_normal = log_normal.cuda()
    # Derivative = backward1(x, log_normal)
    # print(Derivative)
    # x = x * log_normal

    x = lr(x)
    return x


class LI_ESN_internal:

    def __init__(self, n_inputs, n_outputs, n_reservoir=500, W=None, W_in=None,
                 noise=0.001, input_shift=None,
                 input_scaling=None, feedback_scaling=None,
                 teacher_scaling=None, teacher_shift=None,
                 out_activation=identity, inverse_out_activation=identity,
                 random_state=None, time_scale=None):
        # check for proper dimensionality of all arguments and write them down.
        self.n_inputs = n_inputs
        self.n_reservoir = n_reservoir
        self.n_outputs = n_outputs
        self.noise = noise
        self.input_shift = correct_dimensions(input_shift, n_inputs)
        self.input_scaling = correct_dimensions(input_scaling, n_inputs)

        self.teacher_scaling = teacher_scaling
        self.teacher_shift = teacher_shift

        self.out_activation = out_activation
        self.inverse_out_activation = inverse_out_activation
        self.random_state = random_state
        self.time_scale = time_scale
        self.W = W
        self.W_in = W_in.to(torch.double)

        # the given random_state might be either an actual RandomState object,
        # a seed or None (in which case we use numpy's builtin RandomState)
        if isinstance(random_state, np.random.RandomState):
            self.random_state_ = random_state
        elif random_state:
            try:
                self.random_state_ = np.random.RandomState(random_state)
            except TypeError as e:
                raise Exception("Invalid seed: " + str(e))
        else:
            self.random_state_ = np.random.mtrand._rand

    def _scale_inputs(self, inputs):
        """for each input dimension j: multiplies by the j'th entry in the
        input_scaling argument, then adds the j'th entry of the input_shift
        argument."""
        if self.input_scaling is not None:
            inputs = np.dot(inputs, np.diag(self.input_scaling))
        if self.input_shift is not None:
            inputs = inputs + self.input_shift
        return inputs

    def _update(self, state, input_pattern, tanh_a=1, tanh_k=1):
        # leaky integrator model:
        # it can adjust timescales for each neurons.
        # print(self.W.dtype, state.dtype, self.W_in.dtype, input_pattern.dtype)
        # print(state.shape, input_pattern.shape)
        if type(state) == np.ndarray:
            state = torch.tensor(state)
        state = state.unsqueeze(1)
        input_pattern = input_pattern.unsqueeze(1)
        state = state.to(torch.double)

        self.W = self.W.cuda()
        self.W_in = self.W_in.cuda()

        preactivation = (torch.mm(self.W, state) + torch.mm(self.W_in, input_pattern)).cuda()

        self.time_scale = self.time_scale.cuda()
        # state = ((1 - self.time_scale) * state + self.time_scale * torch.tanh(preactivation)).cuda()
        state = ((1 - self.time_scale) * state + self.time_scale * sigmoid_(preactivation, tanh_a, tanh_k)).cuda()

        return state

    def calc_lyapunov_exp(self, inputs, initial_distance, n):
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        states1 = np.zeros((inputs.shape[0], self.n_reservoir))
        states2 = np.zeros((inputs.shape[0], self.n_reservoir))
        transient = min(int(inputs.shape[0] / 10), 100)
        for i in range(1, transient):
            states1[i, :] = self._update(states1[i - 1], inputs[i, :])
        states2[transient - 1, :] = states1[transient - 1, :]
        states2[transient - 1, n] = states2[transient - 1, n] + initial_distance
        gamma_k_list = []
        for k in range(transient, inputs.shape[0]):
            states1[k, :] = self._update(states1[k - 1], inputs[k, :])
            states2[k, :] = self._update(states2[k - 1], inputs[k, :])
            gamma_k = np.linalg.norm(states2[k, :] - states1[k, :])
            gamma_k_list.append(gamma_k / initial_distance)
            states2[k, :] = states1[k, :] + (initial_distance / gamma_k) * (states2[k, :] - states1[k, :])
        lyapunov_exp = np.mean(np.log(gamma_k_list))
        return lyapunov_exp


    def fit(self, inputs, outputs, tanh_a, tanh_k):
        if inputs.ndim < 2:
            inputs = inputs.resize_(len(inputs), 1)
        if outputs.ndim < 2:
            outputs = outputs.resize_(len(outputs), 1)
        inputs_scaled = self._scale_inputs(inputs)
        teachers_scaled = outputs

        # step the reservoir through the given input,output pairs:
        # states = np.zeros((inputs.shape[0], self.n_reservoir))
        # 设置状态为均值为0，方差为0.1的正态分布，使参数更容易收敛。
        with torch.no_grad():
            # setup_seed(2019)
            states = torch.normal(0, 0.1 ** 0.5, (inputs.shape[0], self.n_reservoir)).cuda()
        # states = torch.zeros([inputs.shape[0], self.n_reservoir]).cuda()
        # print("States:{0}".format(states[0][0]))

        for n in range(1, inputs.shape[0]):
            tem = self._update(states[n - 1], inputs_scaled[n, :], tanh_a, tanh_k)
            states[n, :] = tem[:, 0]

        transient = min(int(inputs.shape[0] / 10), 500)  # 原本是100
        extended_states = torch.cat((states, inputs), dim=1)
        # extended_states = np.hstack((states, inputs_scaled))
        pinv = torch.linalg.pinv(extended_states[transient:, :])
        teachers_scaled = teachers_scaled.cuda()
        self.W_out = torch.mm(pinv, teachers_scaled[transient:, :]).T
        # self.W_out = np.dot(np.linalg.pinv(extended_states[transient:, :]), teachers_scaled[transient:, :]).T
        self.W_out = torch.as_tensor(self.W_out)

        # remember the last state for later:
        self.laststate = states[-1, :]
        self.lastinput = inputs[-1, :]
        self.lastoutput = teachers_scaled[-1, :]

        # apply learned weights to the collected states:
        pred_train = torch.mm(extended_states, self.W_out.T)
        # pred_train = np.dot(extended_states, self.W_out.T)
        return pred_train

    def predict(self, inputs, tanh_a, tanh_k, continuation=True):
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        n_samples = inputs.shape[0]

        if continuation:
            laststate = self.laststate
            lastinput = self.lastinput
            lastoutput = self.lastoutput
        else:
            laststate = np.zeros(self.n_reservoir)
            lastinput = np.zeros(self.n_inputs)
            lastoutput = np.zeros(self.n_outputs)

        inputs = torch.as_tensor(inputs)
        # laststate = laststate.T
        laststate = torch.unsqueeze(laststate, dim=1)
        laststate = laststate.T
        states = torch.cat((laststate, torch.zeros((n_samples, self.n_reservoir)).cuda()), dim=0)
        # states = np.vstack([laststate, np.zeros((n_samples, self.n_reservoir))])

        lastoutput = torch.unsqueeze(lastoutput, dim=1)
        lastoutput = lastoutput.T
        outputs = torch.cat((lastoutput, torch.zeros((n_samples, self.n_outputs)).cuda()), dim=0)
        # outputs = np.vstack([lastoutput, np.zeros((n_samples, self.n_outputs))])

        states = torch.as_tensor(states)
        outputs = torch.as_tensor(outputs)
        for n in range(n_samples):
            tem = self._update(states[n, :], inputs[n, :], tanh_a, tanh_k)

            states[n + 1, :] = tem[:, 0]
            # outputs[n + 1, :] = np.dot(self.W_out, np.concatenate([states[n + 1, :], inputs[n, :]]))
            t1 = torch.cat((states[n + 1, :], inputs[n, :]), dim=0)
            t1 = torch.unsqueeze(t1, dim=1)
            wt1 = torch.mm(self.W_out, t1)
            wt1 = wt1.T
            outputs[n + 1:] = wt1

        return self.out_activation(outputs[1:])



def calculate(data_train, label_train, data_test, label_test, p1, p2, n, average_degree, m,
              neg_ratio=0.5, two_way_ratio=0.5, tanh_a=1, tanh_k=1):

    for k in range(1):
        if m != 1:
            W = generate_networks.make_modular_network(n, average_degree, m, p1, p2, neg_ratio, two_way_ratio)
        else:
            W, point = generate_networks.make_ESN(n, p1, neg_ratio, two_way_ratio)

        W = torch.tensor(W, dtype=torch.double)
        W_IN = torch.Tensor(n, 128).uniform_(-0.1, 0.1)  # Set W_in

        # radius = np.max(np.abs(np.linalg.eigvals(W)))  # modify radius
        features = torch.linalg.eigvals(W)
        actual_radius = torch.max(torch.abs(features))
        print("Before spectral_radius:{0}".format(actual_radius))
        W = W * (SPECT_RADIUS / actual_radius)
        # rad = torch.eig(W, eigenvectors=False).eigenvalues
        features = torch.linalg.eigvals(W)
        Now_radius = torch.max(torch.abs(features))
        print("Now spectral_radius:", Now_radius)

        # 构建模块化ESN
        esn = LI_ESN_internal(n_inputs=128,
                              n_outputs=1,
                              n_reservoir=n,
                              W=W,
                              W_in=W_IN,
                              noise=0.00,
                              time_scale=time_scale)

        # 拟合训练集
        with torch.no_grad():
            esn.fit(data_train, label_train, tanh_a, tanh_k).cuda()
        print("Fitting OK!")
        # 预测测试集
        with torch.no_grad():
            prediction = esn.predict(data_test, tanh_a, tanh_k)
        prediction = torch.as_tensor(prediction)
        print("Predicting OK!")
        
        prediction, label_test = list(prediction.cpu().numpy()), list(label_test.cpu().numpy())

        r = Res(prediction, label_test)
        # res1(prediction, label_test)
        

    return r


In [13]:
def save(row, col, result, path):
    '''
    Save result
    '''

    data = openpyxl.load_workbook(path)
    sheetnames = data.sheetnames
    table = data[sheetnames[0]]
    table = data.active  # Default the first sheet
    r, c = row, col
    for i in result:
        table.cell(row, col).value = i
        col += 1
    data.save(xlsx_path)

In [14]:
is_cuda = torch.cuda.is_available()  # Using GPU
print("Is_GPU:", is_cuda)

all_time = time.time()  # Record begin time



# 路径
xlsx_path = 'Result/Base/data.xlsx'  # xlsx路径

# 自变量
esn_paras = {'N': 500, 'Raidus': 0.9, 'LeakyRate': 0.2, 'M': 2,
              'AverageDegree': 500, 'P1': 0.7, 'P2': 0.3, 'TwoWayRate': 0.5,
              'NegRate': 0.5, 'tanh_a': 1, 'tanh_k': 1}

paras = {'row': 4, 'col': 2, 'save_path': 'Result/AAAI Data/state.xlsx'}


p1, p2, m, nodes = 0.1, 0.02, 1, 600
ad, neg, tw = 10, 0.5, 0.5
ta, tk = 1, 0.1
# m_list = np.array([1, 2, 4, 8, 10, 20, 40, 50])  # 模块化数量 Modularity
m_list = np.array([1])
# m_list = np.array([1, 2, 3, 4, 5, 6, 8, 10])
p1_list = np.arange(0.01, 0.51, 0.01)  # 模块内连接概率
p2_list = np.arange(0.02, 0.021, 0.05)  # 模块间连接概率
n_list = np.arange(400, 1001, 200)  # 储层节点数
r_list = np.arange(0.1, 1.01, 0.1)  # 谱半径
l_list = np.arange(0.1, 1.01, 0.1)  # 泄露率
tw_ratio_list = np.arange(1, 1.01, 0.1)  # 双向连接比例
neg_ratio_list = np.arange(1, 1.01, 0.1)  # 负反馈比例
ad_list = np.arange(10, 11, 10)  # 平均度

row, col = 2, 1  # 保存所有指标

data, label = readData()
kf = KFold(n_splits=5, shuffle=False)  # 初始化KFold 5折

for repeat in range(5):
    for esn_paras['N'] in range(500, 501, 100):
        for esn_paras['M'] in [2, 4, 5, 10]:

            res = [0 for ri in range(9)]
            re, order = 0, 1

            start_time = time.time()  # Record One time
            for train_index, test_index in kf.split(data):  # 调用split方法切分数据
                data_train, label_train = data[train_index], label[train_index]
                data_test, label_test = data[test_index], label[test_index]
                len_train, len_test = len(data_train), len(data_test)

                time_scale = torch.ones(esn_paras['N'], 1) * esn_paras['LeakyRate']
                SPECT_RADIUS = esn_paras['Raidus']  # 谱半径

                result = calculate(data_train=data_train,
                                   label_train=label_train,
                                   data_test=data_test,
                                   label_test=label_test,
                                        p1=esn_paras['P1'],
                                         p2=esn_paras['P2'],
                                         n=esn_paras['N'],
                                         average_degree=esn_paras['AverageDegree'],
                                         m=esn_paras['M'],
                                         neg_ratio=esn_paras['NegRate'],
                                         two_way_ratio=esn_paras['TwoWayRate'],
                                         tanh_a=esn_paras['tanh_a'],
                                         tanh_k=esn_paras['tanh_k'],
                                         )

                end_time = time.time() - start_time
                print('***' * 24)
                print('---' * 11 + "Result" + '---' * 11)
                print("第{0}折结果：".format(order))
                print(result)
                order += 1
                print("acc = {0}".format(result[0]))
                print('---' * 24)
                print('***' * 24)


                # save(row, col, result, xlsx_path)

                for i in range(9):
                    res[i] = res[i] + result[i]

            # Get average value
            for i in range(9):
                res[i] /= 5

            save(paras['row'], paras['col'], res, xlsx_path)
            paras['row'] += 3
            # print("N={0},m={1},r={2},a={3},P1={4},P2={5},Neg_ratio={6},Two-way-Ratio={7},Runtime={8} Ad={9}"
            #       .format(esn_paras['N'], m, SPECT_RADIUS, a, p1, p2, neg, tw, round(time.time() - start_time, 2), ad))
            print("五折交叉验证后结果:", res)
            print('*--' * 10)


Is_GPU: True
data_shape:torch.Size([72000, 128]), label_shape:torch.Size([72000, 1])
MESN: m=2 p1=0.7 p2=0.3 Ad=500 NegRatio=0.5 TwoWayRatio=0.5
M-ESN model OK!
Before spectral_radius:20.09221921139597
Now spectral_radius: tensor(0.9000, dtype=torch.float64)
Fitting OK!
Predicting OK!
************************************************************************
---------------------------------Result---------------------------------
第1折结果：
[2695, 3545, 4097, 4063, 0.46930555555555553, 0.5153475393201421, 0.5340431125131441, 0.4979166666666667, 0.46490384615384617]
acc = 2695
------------------------------------------------------------------------
************************************************************************
MESN: m=2 p1=0.7 p2=0.3 Ad=500 NegRatio=0.5 TwoWayRatio=0.5
M-ESN model OK!
Before spectral_radius:20.665007189421612
Now spectral_radius: tensor(0.9000, dtype=torch.float64)
Fitting OK!
Predicting OK!
************************************************************************
--

In [15]:
# 发送邮件
all_time = time.time() - all_time
all_time = strftime("%H:%M:%S", gmtime(all_time))
print("Runtime:{0}".format(all_time))
context = "Runtime:" + str(all_time) + "\nOK!"
sm.send_mail(context)


Runtime:00:34:05
Send mail OK!
Finish!
